In [46]:
from langchain_ollama.llms import OllamaLLM
from langchain_ollama.embeddings import OllamaEmbeddings

MODEL = "llama2"

model = OllamaLLM(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

In [47]:
from langchain_community.document_loaders import PyPDFLoader

# loader = PyPDFLoader('pdfs/test.pdf')
loader = PyPDFLoader('pdfs/test_numerico.pdf')
documents = loader.load()

In [ ]:
documents

In [48]:
import os
from dotenv import load_dotenv

load_dotenv('.env')

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
PINECONE_API_REGION = os.environ.get('PINECONE_API_REGION')

In [49]:
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone.vectorstores import PineconeVectorStore
import time

pc = Pinecone(api_key=PINECONE_API_KEY)
serverless_spec = ServerlessSpec(cloud="aws", region=PINECONE_API_REGION)

index_name = "test-numerico"

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=4096,
        metric="cosine",
        spec=serverless_spec,
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [50]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

if index.describe_index_stats()["total_vector_count"] == 0:
    vector_store.add_documents(documents)

retriever = vector_store.as_retriever()

In [51]:
from langchain.prompts import PromptTemplate

template = """
Respondé a la siguiente pregunta basada en el contexto dado a continuación.
Si no podés responder a la pregunta, respondé con "No tengo una respuesta."

Contexto: {context}

Pregunta: {question}
"""

prompt = PromptTemplate.from_template(template)

In [52]:
import langchain_core.output_parsers

parser = langchain_core.output_parsers.StrOutputParser()

In [53]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [57]:
questions = [
    "Listar todas las secciones del documento",
    "¿Qué pide el ejercicio 6?",
    "¿Cuál es la respuesta al inciso a. del ejercicio 6?",
]

from pprint import pprint

for question in questions:
    pprint(f"Pregunta: {question}")
    pprint(f"Respuesta: {chain.invoke({'question': question})}")
    print()

'Pregunta: Listar todas las secciones del documento'
('Respuesta: A continuación, se presentan todas las secciones del documento:\n'
 '\n'
 '1. Introducción: Discute el objetivo del análisis numérico y los conceptos '
 'básicos que se utilizarán a lo largo del trabajo. (página 0.0)\n'
 '2. Análisis numérico: Define el análisis numérico y sus ventajas en '
 'comparación con otros métodos de resolución de ecuaciones diferenciales. '
 '(página 0.1)\n'
 '3. Modelos lineales y no lineales: Describe los modelos lineales y no '
 'lineales que se pueden resolver mediante el análisis numérico. (página 0.2)\n'
 '4. Métodos de diferencias finitas: Presenta los métodos de diferencias '
 'finitas y sus ventajas en términos de precisión y velocidad de cálculo. '
 '(página 0.3)\n'
 '5. Esquema de diferencias finitas central: Describe el esquema de '
 'diferencias finitas central y su implementación para resolver ecuaciones '
 'diferenciales no lineales. (página 0.4)\n'
 '6. Esquema de diferencias fin

In [54]:
questions = [
    "¿Quiénes son los autores?",
    "¿De qué trata el documento?",
    "¿Cuál es la hipótesis?",
    "¿Cuáles son los objetivos de cada semana?",
]

In [39]:
from pprint import pprint

for question in questions:
    pprint(f"Pregunta: {question}")
    pprint(f"Respuesta: {chain.invoke({'question': question})}")
    pprint()

Question: ¿Quiénes son los autores?
Answer: La pregunta que has proporcionado es una pregunta contextual, ya que no hay suficiente información disponible en el documento para responder de manera precisa. La autoría del documento no está claramente establecida en la pregunta, por lo que no tengo una respuesta definitiva. Sin embargo, si necesitas más información sobre los autores o el documento en general, estaré encantado de ayudarte en el futuro.

Question: ¿De qué trata el documento?
Answer: El documento se centra en investigar las ventajas de Retrieval Augmented Generation (RAG) para mejorar la precisión y calidad de las respuestas generadas por modelos de lenguaje. Se analiza cómo RAG puede optimizar la respuesta a preguntas complejas, especializadas o que requieren información actualizada, superando algunas de las limitaciones inherentes a los modelos de lenguaje tradicionales. Los objetivos del proyecto incluyen clasificar los tipos de preguntas que son adecuadas para el enfoque 

In [40]:
# chain.batch([{"question": q} for q in questions])

In [41]:
# for s in chain.stream([{"question": q} for q in questions]):
#     print(s, end="", flush=True)